# Reinforcement Learning Agent

In [29]:
!pip install imageio
!pip install PILLOW
!pip install pyglet
!pip install pyvirtualdisplay
!pip install dm-acme

In [30]:
import numpy as np
import tensorflow as tf

In [31]:
import os
import sys
sys.path.insert(0, os.path.join(os.getcwd(), '..'))
from Game.AI.Ai import AI
from Game.Board import GameBoard
from Game.AI.ReinforcementAgent import ReinforcementAI

In [19]:
%%time
"""Training the agent"""

import random

alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting
all_epoch = []
all_penalties = []

for i in range(1, 100001):
    state = Gameboard()
    